# Challenge
We have built a cellular automata with 64 bit steps and obeys Wolfram rule 126, it's boundary condition wraps around so that the last bit is a neighbor of the first bit. Below you can find a special step we chose from the automata.

The flag is encrypted with AES256-CBC, the encryption key is the previous step that generates the given step. Your task is to reverse the given step and find the encryption key.

Example decryption with 32 bit steps:

```
echo "404c368b" > /tmp/plain.key; xxd -r -p /tmp/plain.key > /tmp/enc.key

echo "U2FsdGVkX18+Wl0awCH/gWgLGZC4NiCkrlpesuuX8E70tX8t/TAarSEHTnpY/C1D" | openssl enc -d -aes-256-cbc -pbkdf2 -md sha1 -base64 --pass file:/tmp/enc.key
```

(Again I could not implement `-pbkdf2`. I wonder how the CTF kids do it.)

Examples of 32 bit steps, reverse_rule126 in the example yields only one of the multiple values.
```
rule126('deadbeef') = 73ffe3b8 | reverse_rule126('73ffe3b8') = deadbeef
rule126('73ffe3b8') = de0036ec | reverse_rule126('de0036ec') = 73ffe3b8
rule126('de0036ec') = f3007fbf | reverse_rule126('f3007fbf') = de0036ec
```

### Flag (base64)
`U2FsdGVkX1/andRK+WVfKqJILMVdx/69xjAzW4KUqsjr98GqzFR793lfNHrw1Blc8UZHWOBrRhtLx3SM38R1MpRegLTHgHzf0EAa3oUeWcQ=`

### Obtained step (in hex)
`66de3c1bf87fdfcf`

In [211]:
def encode_bin(hex_str):
    bin_str = bin(int(hex_str, 16))[2:]
    bin_str = "0"*(len(hex_str)*4-len(bin_str)) + bin_str
    return bin_str

In [212]:
def decode_bin(bin_str):
    hex_str = hex(int(bin_str, 2))[2:]
    hex_str = "0"*int(len(bin_str)/4-len(hex_str)) + hex_str
#     print()
    return hex_str

In [239]:
print(decode_bin(encode_bin("00000000000")))
print(decode_bin(encode_bin("000000000000")))
print(decode_bin(encode_bin("0000000000000")))
print(decode_bin(encode_bin("00000000000000")))

00000000000
000000000000
0000000000000
00000000000000


In [229]:
def next_step(bin_str):
    print("next bin_str inp: ", bin_str)
    bin_str = bin_str[-1:] + bin_str + bin_str[:1]
    arr = []
    for i,j,k in zip(bin_str[2:],bin_str[1:-1],bin_str[:-2]):
        if i==j==k:
            arr.append(0)
        else:
            arr.append(1)
    res = "".join([str(i) for i in arr])
    print("next bin_str out: ", res)
    return res

In [230]:
comb = ["0","1"]
tri_map = {}
for a in comb:
    for b in comb:
        for c in comb:
            for d in comb:
                for e in comb:
                    next_val = next_step("".join([a,b,c,d,e]))[1:-1]
                    if not next_val in tri_map:
                        tri_map[next_val] = [b+c+d]
                    else:
                        tri_map[next_val].append(b+c+d)
                    print(tri_map)
                    print(next_val)
print(tri_map)

next bin_str inp:  00000
next bin_str out:  00000
{'000': ['000']}
000
next bin_str inp:  00001
next bin_str out:  10011
{'001': ['000'], '000': ['000']}
001
next bin_str inp:  00010
next bin_str out:  00111
{'001': ['000'], '011': ['001'], '000': ['000']}
011
next bin_str inp:  00011
next bin_str out:  10111
{'001': ['000'], '011': ['001', '001'], '000': ['000']}
011
next bin_str inp:  00100
next bin_str out:  01110
{'001': ['000'], '011': ['001', '001'], '111': ['010'], '000': ['000']}
111
next bin_str inp:  00101
next bin_str out:  11111
{'001': ['000'], '011': ['001', '001'], '111': ['010', '010'], '000': ['000']}
111
next bin_str inp:  00110
next bin_str out:  01111
{'001': ['000'], '011': ['001', '001'], '111': ['010', '010', '011'], '000': ['000']}
111
next bin_str inp:  00111
next bin_str out:  11101
{'001': ['000'], '011': ['001', '001'], '111': ['010', '010', '011'], '110': ['011'], '000': ['000']}
110
next bin_str inp:  01000
next bin_str out:  11100
{'001': ['000'], '011': 

In [231]:
def prev_step(bin_str):
    print("prev bin_str inp: ", bin_str)
    bin_str = bin_str[:1] + bin_str + bin_str[-1:]
    arr = []
    
    for i,j,k in zip(bin_str[2:],bin_str[1:-1],bin_str[:-2]):
        possibilites = tri_map["".join([i,j,k])]
        print(possibilites)
        arr.append(possibilites)
    
    res = arr[0][0][1:]
    print(res)
    for ar in arr[1:]:
        for a in ar:
            if a[0] == res[-1]:
                print(res)
                res += a[2]
                break
        
    print("prev bin_str out: ", res[:-1])
    return res[:-1]

In [232]:
def rule126(hex_str):
    bin_str = encode_bin(hex_str)
    bin_str = next_step(bin_str)
    hex_str = decode_bin(bin_str)
    return hex_str

In [233]:
def reverse_rule126(hex_str):
    bin_str = encode_bin(hex_str)
    bin_str = prev_step(bin_str)
    hex_str = decode_bin(bin_str)
    return hex_str

In [236]:
# rule126('deadbeef') = 73ffe3b8 | reverse_rule126('73ffe3b8') = deadbeef
# rule126('73ffe3b8') = de0036ec | reverse_rule126('de0036ec') = 73ffe3b8
# rule126('de0036ec') = f3007fbf | reverse_rule126('f3007fbf') = de0036ec
print(rule126('deadbeef'))
print(rule126('73ffe3b8'))
print(rule126('de0036ec'))  # wrong

next bin_str inp:  1101111010101101101100010010100100111000000010010001111011101111
next bin_str out:  0111001111111111111110111111111111101100000111111011001110111000
73fffbffec1fb3b8
next bin_str inp:  01110011111111111110001110111000
next bin_str out:  11011110000000000011011011101100
de0036ec
next bin_str inp:  11011110000000000011011011101100
next bin_str out:  11110011000000000111111110111111
f3007fbf


In [238]:
# rule126('deadbeef') = 73ffe3b8 | reverse_rule126('73ffe3b8') = deadbeef
# rule126('73ffe3b8') = de0036ec | reverse_rule126('de0036ec') = 73ffe3b8
# rule126('de0036ec') = f3007fbf | reverse_rule126('f3007fbf') = de0036ec

print(rule126('deadbeef'))
preimage = reverse_rule126(rule126('deadbeef'))
print(preimage)
print(rule126(preimage))

next bin_str inp:  11011110101011011011111011101111
next bin_str out:  01110011111111111110001110111000
73ffe3b8
next bin_str inp:  11011110101011011011111011101111
next bin_str out:  01110011111111111110001110111000
prev bin_str inp:  01110011111111111110001110111000
['111', '000']
['011', '100', '011', '100']
['010', '010', '011', '100', '101', '101', '110', '110', '001', '001', '010', '010', '011', '100', '101', '101']
['001', '001', '110', '110']
['000', '111']
['111', '000']
['011', '100', '011', '100']
['010', '010', '011', '100', '101', '101', '110', '110', '001', '001', '010', '010', '011', '100', '101', '101']
['010', '010', '011', '100', '101', '101', '110', '110', '001', '001', '010', '010', '011', '100', '101', '101']
['010', '010', '011', '100', '101', '101', '110', '110', '001', '001', '010', '010', '011', '100', '101', '101']
['010', '010', '011', '100', '101', '101', '110', '110', '001', '001', '010', '010', '011', '100', '101', '101']
['010', '010', '011', '100', '101'

In [ ]:
# Unfortunately, I cannot find a single preimage that provides the next step.
# Not even for the characters in between.

In [ ]:
# I expect a binary tree will be formed, and then the search is within brute force limits.
# I really need to do my homework, so this should be it.